In [1]:
from os import makedirs

from numpy import expand_dims, zeros, ones
from numpy.random import randn, randint

from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from keras.initializers import RandomNormal

from matplotlib import pyplot

In [2]:
# define Generator

def define_generator(latent_dim):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    
    # define model
    model = Sequential()
    n_nodes = 128*7*7
    
    model.add(Dense(n_nodes, kernel_initializater=init, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    
    # upsample 14x14x128
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializater=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    # upsample 28x28x128
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializater=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    # output 28x28x1
    model.add(Conv2D(1, (7, 7), activation='tanh', padding='same', kernel_initializater=init))
    
    return model

In [4]:
# define discriminator

def define_discriminator(in_shape=(28, 28, 1)):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    
    # define model
    model = Sequential()
    
    # downsample 14x14
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializater=init, input_shape=in_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    # downsample 7x7
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializater=init, input_shape=in_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    # classifier
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model